<a href="https://colab.research.google.com/github/spatank/CIS-700/blob/master/Project/log_odds_corpus_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/Drive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/Drive


In [0]:
import os
os.chdir('/content/Drive/My Drive/CIS-700/')

In [3]:
!ls

Data				  log_odds_corpus_level.ipynb
draco_verb_network_ff.graphml	  log_odds.ipynb
global_A_large.mat		  networks.ipynb
global_verb_network_ff.graphml	  __pycache__
harry_verb_network_ff.graphml	  ron_verb_network_ff.graphml
hermione_verb_network_ff.graphml  util.py
hp_narrative_chains.py		  voldemort_verb_network_ff.graphml


In [4]:
!ls Data

04212020
04292020
Canon
hpcanon_sr_narrative_chains_counts_NNPs.txt
hpc_raw_text.txt
hpff_raw_text_reduced.txt
hpff_raw_text.txt
hpff_sr_narrative_chains_counts_NNPs_new.txt


In [5]:
import collections
import nltk
import numpy as np

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
canon_word_counts = collections.Counter()

with open('Data/hpc_raw_text.txt', encoding = 'utf-8', errors = 'ignore') as f:
  for line in f:
    words = nltk.word_tokenize(line.strip())
    words_no_punctuation = [word.lower() for word in words if word.isalnum()]
    canon_word_counts.update(words_no_punctuation)

In [0]:
from itertools import islice

corpus_path = 'Data/hpff_raw_text.txt'
file = open(corpus_path, encoding = 'UTF-8', errors = 'ignore').readlines()
file_len = len(file)
num_lines_keep = round(0.10 * file_len)
num_lines_discard = file_len - num_lines_keep
lines_in_sets = [num_lines_keep, num_lines_discard]
temp = iter(file) 
splits = [list(islice(temp, 0, ele)) for ele in lines_in_sets] 
# split 0 is data to keep
with open('Data/hpff_raw_text_reduced.txt', 'w') as f:
  for line in splits[0]:
    f.write('%s\n' % line)

In [0]:
fan_fiction_word_counts = collections.Counter()

with open('Data/hpff_raw_text_reduced.txt', encoding = 'utf-8', errors = 'ignore') as f:
  for line in f:
    words = nltk.word_tokenize(line.strip())
    words_no_punctuation = [word.lower() for word in words if word.isalnum()]
    fan_fiction_word_counts.update(words_no_punctuation)

In [9]:
keys_a = set(canon_word_counts.keys())
keys_b = set(fan_fiction_word_counts.keys())
intersection = keys_a & keys_b # '&' operator is used for set intersection
print(len(intersection))

19463


What words appear more often in fan fiction than the canonical Harry Potter books?

In [20]:
total_num_words_in_fan_fiction = sum(fan_fiction_word_counts.values())
total_num_words_in_canon = sum(canon_word_counts.values())

log_odds_data = {}

for word in canon_word_counts.keys():
  
  count_canon = canon_word_counts[word]
  count_fan_fiction = fan_fiction_word_counts[word]
  
  log_prob_canon = np.log(count_canon) - np.log(total_num_words_in_canon)
  log_prob_fan_fiction = np.log(count_fan_fiction) - np.log(total_num_words_in_fan_fiction)
  

  if log_prob_fan_fiction != 0:
    log_odds = log_prob_fan_fiction - log_prob_canon
    log_odds_data[word] = log_odds

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log
  if sys.path[0] == '':


In [21]:
log_odds_tuples = [(word, log_odds) for word, log_odds in log_odds_data.items()]
log_odds_tuples = sorted(log_odds_tuples, key=lambda x: x[1], reverse=True)

for i in range(0, 50):
  print(log_odds_tuples[i])

('scorpius', 5.50767680151591)
('guy', 4.87872015645234)
('alright', 4.801797083271262)
('haired', 4.610930665714724)
('figured', 4.507773400741616)
('dorm', 4.474203815018763)
('emma', 4.362182951238399)
('hugo', 4.3188274752575)
('amy', 4.314108521323547)
('anyways', 4.241412163135381)
('shit', 4.2333003566000045)
('colour', 3.883710788518899)
('characters', 3.8811400930157998)
('uh', 3.8484360665979906)
('victoire', 3.7774941084252127)
('iris', 3.7767800776915568)
('trio', 3.7164232771506587)
('evan', 3.67141956203373)
('defence', 3.617616856040753)
('commented', 3.604125926299739)
('marlene', 3.5870022202211462)
('alice', 3.5820967133417216)
('crap', 3.5748387340279493)
('honey', 3.566059046375903)
('dorcas', 3.5607539941462107)
('wards', 3.5572015925418423)
('arse', 3.5301485450701318)
('ok', 3.430257505216842)
('kisses', 3.4262089166908414)
('blaise', 3.414648094289765)
('review', 3.40031883285398)
('besides', 3.3763025110944316)
('catherine', 3.359820590157657)
('dating', 3.3481

# Canon with Standard English

In [14]:
!pip install wordfreq
import wordfreq

     |████████████████████████████████| 32.8MB 119kB/s 
     |████████████████████████████████| 4.9MB 42.1MB/s 
     |████████████████████████████████| 624kB 42.2MB/s 
     |████████████████████████████████| 276kB 43.8MB/s 
  Created wheel for wordfreq: filename=wordfreq-2.2.2-cp36-none-any.whl size=32816665 sha256=bac08424126b2e040b916de005df39a74e4c3f286178f3cd118a676aec87ff40
  Stored in directory: /root/.cache/pip/wheels/33/2e/fc/e447859743f61cdf41873a5bcc11300c05fbd27631aea984e1
  Created wheel for langcodes: filename=langcodes-2.0.0-cp36-none-any.whl size=5044047 sha256=3ed0402d66f3e76d05e2b17bef27e6d0cf6a0e4d1b38edb5a27d28fde3947bb6
  Stored in directory: /root/.cache/pip/wheels/c9/11/90/c7bba8118f3674d75e1457537635266a12538cf622a4684bb2
  Created wheel for regex: filename=regex-2018.2.21-cp36-cp36m-linux_x86_64.whl size=552310 sha256=3ee6fd822e79d107a2fef349efaa9cef8b71477990811199de2c320c904b919a
  Stored in directory: /root/.cache/pip/wheels/6b/c9/cf/230425cdd343d6b98e8da5a58

In [0]:
total_num_words_in_canon = sum(canon_word_counts.values())

log_odds_data = {}

for word in canon_word_counts.keys():
  count_canon = canon_word_counts[word]

  log_prob_canon = np.log(count_canon) - np.log(total_num_words_in_canon)
  log_prob_english = wordfreq.zipf_frequency(word, lang="en")

  if log_prob_english != 0:
    log_odds = log_prob_canon - log_prob_english
    log_odds_data[word] = log_odds

In [17]:
log_odds_tuples = [(word, log_odds) for word, log_odds in log_odds_data.items()]
log_odds_tuples = sorted(log_odds_tuples, key=lambda x: x[1], reverse=True)

for i in range(0, 50):
  print(log_odds_tuples[i])

('hermione', -8.553356963296721)
('hagrid', -8.755619612117385)
('harry', -8.833787911198272)
('dumbledore', -8.84744015807585)
('weasley', -9.130941622583435)
('malfoy', -9.207863910463326)
('snape', -9.251193306242097)
('harrys', -9.291742064769004)
('ron', -9.394647261463543)
('slughorn', -9.534338152023214)
('mcgonagall', -9.549610976581542)
('kreacher', -9.622336642291202)
('voldemort', -9.812289512555745)
('umbridge', -9.87077303339843)
('dursleys', -9.888013929752669)
('wand', -9.898043790903325)
('lupin', -9.991193221767068)
('griphook', -9.995416533186516)
('goyle', -10.004590909420527)
('pomfrey', -10.010253390232512)
('gryffindor', -10.07226512797336)
('filch', -10.11488924532895)
('wormtail', -10.117734460734736)
('scabbers', -10.130342183403194)
('ginny', -10.14061810509459)
('rons', -10.148417883578317)
('crookshanks', -10.150835304375411)
('sirius', -10.17844784735874)
('hogwarts', -10.185652334844406)
('said', -10.274874839756961)
('scrimgeour', -10.314864254023686)
('d

# Fan Fiction with Standard English

In [0]:
total_num_words_in_fan_fiction = sum(fan_fiction_word_counts.values())

log_odds_data = {}

for word in fan_fiction_word_counts.keys():
  count_fan_fiction = fan_fiction_word_counts[word]

  log_prob_fan_fiction = np.log(count_fan_fiction) - np.log(total_num_words_in_fan_fiction)
  log_prob_english = wordfreq.zipf_frequency(word, lang="en")

  if log_prob_english != 0:
    log_odds = log_prob_fan_fiction - log_prob_english
    log_odds_data[word] = log_odds

In [19]:
log_odds_tuples = [(word, log_odds) for word, log_odds in log_odds_data.items()]
log_odds_tuples = sorted(log_odds_tuples, key=lambda x: x[1], reverse=True)

for i in range(0, 50):
  print(log_odds_tuples[i])

('hermione', -8.933544474984508)
('remus', -9.450006376913379)
('ginny', -9.46480384054967)
('draco', -9.472216011416899)
('sirius', -9.500748395124791)
('malfoy', -9.660939644119985)
('harry', -9.84429530706121)
('weasley', -10.016087966340125)
('dumbledore', -10.059386108895115)
('snape', -10.226584996194157)
('pomfrey', -10.268017451460269)
('mcgonagall', -10.274376769421487)
('ron', -10.302354345524574)
('lily', -10.32246087496647)
('albus', -10.32789649242774)
('hogwarts', -10.346175047028712)
('gryffindor', -10.388617638049407)
('voldemort', -10.423256454010946)
('slytherins', -10.549085599609398)
('scorpius', -10.553014510109037)
('she', -10.55924394137414)
('her', -10.567611971978383)
('severus', -10.611834698062976)
('slytherin', -10.620285057261597)
('wand', -10.648206295309455)
('muggle', -10.6812717337462)
('nodded', -10.698496239523761)
('quidditch', -10.777603670275038)
('he', -10.78328324705663)
('padfoot', -10.799279148489566)
('i', -10.805122244364352)
('auror', -10.81